# Pre-trained Model

In [1]:
!git clone https://github.com/zhangyi-3/KBNet.git
%cd KBNet

Cloning into 'KBNet'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 144 (delta 34), reused 133 (delta 26), pack-reused 0
Receiving objects: 100% (144/144), 516.58 KiB | 2.36 MiB/s, done.
Resolving deltas: 100% (34/34), done.
/content/KBNet


In [2]:
!pip install -r requirements.txt
!pip install --upgrade --no-cache-dir gdown
!python3 setup.py develop --no_cuda_ext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        *******

In [3]:
import gdown
gdown.download('https://drive.google.com/uc?id=1dnXSKKc7wMjEYIO_bEHI0D43_HRWzJS4', "./Denoising/pretrained_models/", quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1dnXSKKc7wMjEYIO_bEHI0D43_HRWzJS4
From (redirected): https://drive.google.com/uc?id=1dnXSKKc7wMjEYIO_bEHI0D43_HRWzJS4&confirm=t&uuid=72c1db7d-aa6e-4074-9304-48da1f8e6922
To: /content/KBNet/Denoising/pretrained_models/KBNet.zip
100%|██████████| 479M/479M [00:01<00:00, 326MB/s]


'./Denoising/pretrained_models/KBNet.zip'

In [4]:
!unzip -q Denoising/pretrained_models/KBNet.zip -d Denoising/pretrained_models/
!rm -rf Denoising/pretrained_models/KBNet.zip

In [5]:
%cd /content/KBNet

/content/KBNet


In [6]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.6 MB/s eta 0:00:00


In [7]:
import os
import yaml
import numpy as np
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from tqdm import tqdm
from skimage import img_as_ubyte
from natsort import natsorted
from glob import glob

from basicsr.models.archs.kbnet_s_arch import KBNet_s
from basicsr.utils.util import patch_forward

try:
    from yaml import CLoader as Loader
except ImportError:
    from yaml import Loader

In [8]:
def load_img(filepath):
    return cv2.cvtColor(cv2.imread(filepath), cv2.COLOR_BGR2RGB)

def save_img(filepath, img):
    cv2.imwrite(filepath,cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

def load_gray_img(filepath):
    return np.expand_dims(cv2.imread(filepath, cv2.IMREAD_GRAYSCALE), axis=2)

def save_gray_img(filepath, img):
    cv2.imwrite(filepath, img)

# Dataset download

In [9]:
gdown.download('https://drive.google.com/uc?id=1UtsutsiIQQXDEEsqjsQcjkwq4f5zLFBc', '/content/',quiet=False)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1UtsutsiIQQXDEEsqjsQcjkwq4f5zLFBc
From (redirected): https://drive.google.com/uc?id=1UtsutsiIQQXDEEsqjsQcjkwq4f5zLFBc&confirm=t&uuid=7b7fefb4-ca1d-4f6e-bd97-146d0bc56f48
To: /content/DenoisedImages.zip
100%|██████████| 651M/651M [00:02<00:00, 317MB/s]


'/content/DenoisedImages.zip'

In [10]:
!unzip -q /content/DenoisedImages.zip -d /content
!rm -rf /content/DenoisedImages.zip

In [11]:
noised_images = [f for f in os.listdir('/content/') if 'noised' in f]
noised_images

['noised_gray_25_sun',
 'noised_25_sun',
 'noised_50_sun',
 'noised_gray_15_sun',
 'noised_gray_50_sun',
 'noised_15_sun']

# Setup
We changed the batch setup:
```ymal
# general settings
name: gaum-kb333
model_type: ImageCleanModel
scale: 1
num_gpu: 4
manual_seed: 100

mini_batch_sizes: [4,3,2,1,1] # Batch size per gpu
iters: [92000,64000,48000,36000,36000,24000]
gt_size: 384   # Max patch size for progressive training
gt_sizes: [128,160,192,256,320,384]  # Patch sizes for progressive training.
```

In [ ]:
yaml_file = '/content/KBNet/Denoising/Options/sidd.yml'
x = yaml.load(open(yaml_file, mode='r'), Loader=Loader)
pth_path = x['path']['pretrain_network_g']
print('**', yaml_file, pth_path)

s = x['network_g'].pop('type')

model_restoration = eval(s)(**x['network_g'])

checkpoint = torch.load('/content/KBNet/Denoising/' + pth_path)
model_restoration.load_state_dict(checkpoint['model'])
print("===>Testing using weights: ")
model_restoration.cuda()
model_restoration = nn.DataParallel(model_restoration)
model_restoration.eval()

In [14]:
import os
os.makedirs('/content/KBNetResults', exist_ok=True)

In [17]:
"""img_path = '/content/noised_15_sun/image_0.png'
torch.cuda.ipc_collect()
torch.cuda.empty_cache()
img = load_img(img_path)
img = np.float32(img) / 255.
img = torch.from_numpy(img).permute(2, 0, 1)
input_ = img.unsqueeze(0).cuda()

restored = patch_forward(input_, model_restoration, skip=skip, padding=padding)
restored = torch.clamp(restored, 0, 1).cpu().detach().permute(0, 2, 3, 1).squeeze(0).numpy()
restored = img_as_ubyte(restored)"""

In [19]:
from PIL import Image

factor = 8
skip, padding = 1024, 64

def KBNetDenoise(dataset):
  print(f"Read {dataset}...")
  folder_name = f'/content/KBNetResults/{dataset}'
  os.makedirs(folder_name, exist_ok=True)
  dataset_path_lst = os.listdir(f'/content/{dataset}')
  for index, img_name in tqdm(enumerate(dataset_path_lst), total=len(dataset_path_lst)):
    img_path = f'/content/{dataset}/{img_name}'
    try:
      torch.cuda.ipc_collect()
      torch.cuda.empty_cache()
      img = load_img(img_path)
      img = np.float32(img) / 255.
      img = torch.from_numpy(img).permute(2, 0, 1)
      input_ = img.unsqueeze(0).cuda()

      restored = patch_forward(input_, model_restoration, skip=skip, padding=padding)
      restored = torch.clamp(restored, 0, 1).cpu().detach().permute(0, 2, 3, 1).squeeze(0).numpy()
      restored = img_as_ubyte(restored)

      # Export
      img = Image.fromarray(restored)
      export_img_path = os.path.join(folder_name, img_name)
      img.save(export_img_path)
    except:
      continue
  print(f"Done {dataset}...")

In [21]:
KBNetDenoise("noised_25_sun")
KBNetDenoise("noised_gray_25_sun")
KBNetDenoise("noised_50_sun")
KBNetDenoise("noised_gray_15_sun")
KBNetDenoise("noised_gray_50_sun")
KBNetDenoise("noised_15_sun")

Read noised_25_sun...


100%|██████████| 80/80 [05:40<00:00,  4.26s/it]


Done noised_25_sun...
Read noised_gray_25_sun...


100%|██████████| 80/80 [05:40<00:00,  4.26s/it]


Done noised_gray_25_sun...
Read noised_50_sun...


...

Done noised_gray_15_sun...
Read noised_gray_50_sun...


100%|██████████| 80/80 [05:34<00:00,  4.19s/it]


Done noised_gray_50_sun...
Read noised_15_sun...


100%|██████████| 80/80 [05:38<00:00,  4.23s/it]

Done noised_15_sun...


In [ ]:
"""for folder in os.listdir('/content/KBNetResults/noised_25_sun'):
  plt.imshow(plt.imread(f'/content/KBNetResults/noised_25_sun/{folder}'))
  plt.show()"""

In [23]:
'1'

'1'

In [22]:
# Export
from google.colab import files

for folder in os.listdir('/content/KBNetResults'):
    folder_path = f'/content/KBNetResults/{folder}'
    zip_command = f'zip -r "{folder_path}.zip" "{folder_path}"'
    os.system(zip_command)
    files.download(f'/content/KBNetResults/{folder}.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>